In [26]:
import sqlite3

connexion = sqlite3.connect('vivino.db')
cursor = connexion.cursor()




In [47]:
query1 = """
SELECT COUNT(wineries.name)
FROM wineries
WHERE EXISTS (
    SELECT 1
    FROM wines
    WHERE wineries.name = wines.name
);

"""
cursor.execute(query1)
cursor.fetchall()

#the number of matches equals to number of wines names and wineries names, therefore, they match

[(1020,)]

In [79]:
#show the wines that have ranking 1 

query2 = """
SELECT DISTINCT wines.name, wines.ratings_average, wines.ratings_count
FROM wines
JOIN vintages ON wines.id = vintages.wine_id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
WHERE vintage_toplists_rankings.rank = 1 AND vintage_toplists_rankings.previous_rank = 1
GROUP BY toplists.name
ORDER BY wines.ratings_average DESC, wines.ratings_count DESC
LIMIT 3
"""

cursor.execute(query2)
cursor.fetchall()

[('Cabernet Sauvignon', 4.8, 2941),
 ('Toscana', 4.7, 16284),
 ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 4.7, 9564)]

In [82]:
#show the wines that have ranking 1 

# guaranteed_rating is the 95% percentile of the rating, based on the sample size
# guaranteed_rating := rating - 1.96 * sqrt(rating*(1-rating)/count)
# https://www.omnicalculator.com/statistics/sampling-error

query2 = """
SELECT DISTINCT wines.name, wines.ratings_average, wines.ratings_count, 5.0*(((wines.ratings_average/5.0) - 1.96 * SQRT((wines.ratings_average/5.0)*(1.0-(wines.ratings_average/5.0))/wines.ratings_count))) AS guaranteed_rating
FROM wines
JOIN vintages ON wines.id = vintages.wine_id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
WHERE vintage_toplists_rankings.rank = 1 AND vintage_toplists_rankings.previous_rank = 1
GROUP BY toplists.name
ORDER BY guaranteed_rating DESC
LIMIT 3
"""

cursor.execute(query2)
cursor.fetchall()

[('Cabernet Sauvignon', 4.8, 2941, 4.76458851205599),
 ('Toscana', 4.7, 16284, 4.6817616695226825),
 ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne',
  4.7,
  9564,
  4.676201704983944)]

In [70]:
query3 = """
SELECT DISTINCT name
FROM toplists
"""
cursor.execute(query3)
result = cursor.fetchall()
awards = []
for i in range(len(result)):
    awards.append(result[i][0])

print (awards)



["Vivino's 2016 Wine Style Awards: Italian Barbaresco", "Vivino's 2016 Wine Style Awards: Italian Barolo", "Vivino's 2016 Wine Style Awards: Italian Bolgheri", "Vivino's 2017 Wine Style Awards: Italian Amarone", "Vivino's 2017 Wine Style Awards: Italian Barolo", "Vivino's 2017 Wine Style Awards: Italian Bolgheri", "Vivino's 2018 Wine Style Awards: Italian Amarone", "Vivino's 2018 Wine Style Awards: Italian Barbaresco", "Vivino's 2018 Wine Style Awards: Italian Barolo", "Vivino's 2018 Wine Style Awards: Italian Bolgheri", "Vivino's 2019 Wine Style Awards: Italian Barbaresco", "Vivino's 2019 Wine Style Awards: Italian Barolo", "Vivino's 2019 Wine Style Awards: Italian Bolgheri", "Vivino's 2020 Wine Style Awards: Italian Amarone", "Vivino's 2020 Wine Style Awards: Italian Barbaresco", "Vivino's 2020 Wine Style Awards: Italian Barolo", "Vivino's 2020 Wine Style Awards: Italian Bolgheri", "Vivino's 2016 Wine Style Awards: Italian Brunello", "Vivino's 2016 Wine Style Awards: Central Italy Re